# MODERN PORTFOLIO THEORY & CAPITAL ASSET PRICING MODEL

## 0) Set-up

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns

In [2]:
from library.main import Portfolio

In [3]:
portfolio = Portfolio()

In [ ]:
sp500 =portfolio.extract_tickers()
sp500

In [ ]:
# Get the first 100 tickers
list_of_tickers = sp500["Symbol"][:3].tolist()
list_of_tickers

In [6]:
start_date = "2000-01-01"
end_date = "2019-12-31"

In [ ]:
portfolio.update_tickers(list_of_tickers, start_date, end_date)

In [8]:
#portfolio.analyze_securities()

Now lets observe the expected returns if we were to invest solely in a single asset

In [ ]:
portfolio.data_extractor.analayze_single_security_returns()

In [ ]:
len(list(portfolio.securities.keys()))

In [ ]:
portfolio.plots.plot_results(include_individual_securities=True)

As you will be able to observe in our library, the way we extract returns is by levearing yfinance's dataframe column 'Adj close'. It stands for adjusted close (price) and it account for the splits and dividends over time. We then compute the percentage change over time of this returns (current value and prior). Note these measures are taken on a daily basis. If you are interested into reading more about this way of computing the returns, you can [read yahoo's blog](https://help.yahoo.com/kb/SLN28256.html#:~:text=What%20is%20the%20adjusted%20close%3F).

In accordance with the CER (cosntant expected return) model -- returns follow a i.i.d normal distribution we need to check the fitting of such distribution and alert of possible violations of this assumption.

<hr>

## 1) Extracting matrices 

In [11]:
portfolio.data_extractor.compute_matrices()

In [ ]:
portfolio.data_extractor.SIGMA

In [ ]:
portfolio.data_extractor.SIGMA_INV

In [ ]:
portfolio.data_extractor.MU

In [ ]:
portfolio.data_extractor.ONE_VECTOR

We want to smooth out noise, thus we can cleverly apply PCA to the Sigma matrix. This will significanlty improve the robustness of our model. This is accomplished by making the model less sensitive to estimation errors in the original sigma

In [ ]:
portfolio.data_extractor.pca_shrinkage(var_threshold=0.90)

In [17]:
# Lets update the class with the new computed data
portfolio.update_data()

# 2) Global Minimum Variance Portfolio
A baseline for our modelling in the risk-return space

In [18]:
results_dict = portfolio.compute_global_minimum_variance_portfolio()

In [ ]:
results_dict["weights"], results_dict["weights"].sum()

In [ ]:
results_dict["expected_return"], results_dict["expected_variance"]

# 3) Efficient Portfolios
Note we do not restrict to long positions on the securities. We allow for shorting

In [ ]:
df = portfolio.compute_efficient_frontier()
df

In [ ]:
portfolio.plots.plot_results(include_gmvp=True, include_efficient_frontier=True, include_individual_securities=True)

# 4) Capital Asset Pricing Model

In [ ]:
annual_risk_free_rate = 0.02
daily_risk_free_rate = (1 + annual_risk_free_rate)**(1/252) - 1 # Formula (assuming 252 days of trading)
daily_risk_free_rate

In [24]:
results_dict = portfolio.compute_sharpe_portfolio(daily_risk_free_rate)

In [ ]:
results_dict["weights"]

In [ ]:
results_dict["expected_return"]

In [ ]:
results_dict["expected_variance"]

In [ ]:
print(f'Daily Sharpe ratio: {results_dict["sharpe_ratio"]} \nAnnualized Sharpe ratio: {results_dict["sharpe_ratio"] * np.sqrt(252)}')

In [ ]:
portfolio.plots.plot_results(include_gmvp=True, include_efficient_frontier=True, include_sharpe=True, include_individual_securities=True)